# Preprocessing

In [ ]:
from mne.preprocessing import compute_bridged_electrodes, interpolate_bridged_electrodes
from mne.viz import plot_bridged_electrodes

from eeg_cybersickness.io import read_raw
from eeg_cybersickness.utils.path import get_derivative_stem

In [ ]:
root = "/mnt/Isilon/9003_CBT_HNP_MEEG/projects/project_cybersickness/data"
participant = 57
session = 1
raw = read_raw(root, participant, session)
derivative_stem = get_derivative_stem(root, participant, session)

## Remove unused electrodes

The mastoids electrodes M1, M2 and the EOG droplead electrode were not connected.

In [ ]:
channels_to_drop = [ch for ch in ("M1", "M2", "EOG") if ch in raw.ch_names]
if len(channels_to_drop) != 0:
    raw.drop_channels(channels_to_drop)

## Look for gel-bridges

If possible, the bridges are repaired by interpolation. If not, they are kept as bads in `raw.info["bads"]`.

In [ ]:
raw.set_montage("standard_1020")
bridged_idx, ed_matrix = compute_bridged_electrodes(raw)
%matplotlib inline
_ = plot_bridged_electrodes(
    raw.info,
    bridged_idx,
    ed_matrix,
    title="Bridged Electrodes",
    topomap_args=dict(vlim=(0, 5)),
)

In [ ]:
interpolate_bridged_electrodes(raw, bridged_idx)

In [ ]:
raw.set_montage(None)

## Filter

In [ ]:
raw.filter(
    l_freq=1.,
    h_freq=30.,
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

## Annotate bad channels and segments

In [ ]:
%matplotlib qt
raw.plot()

## Save RAW before ICA decomposition

In [ ]:
fname = derivative_stem.with_name(f"{derivative_stem.name}-raw.fif")
raw.save(fname)